<a href="https://colab.research.google.com/github/GorokhovSemyon/TextClassification/blob/main/QASystem_Ru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


В этом ноутбуке рассматривается типичное решение - RAG, для которого мы будем использовать модель с открытым исходным кодом и векторную базу данных Chroma DB. **Также будет интегрирована система семантического кэширования, которая будет хранить различные пользовательские запросы и решать, следует ли генерировать запрос, обогащенный информацией из векторной базы данных, или из базы данных с открытым исходным кодом. тайник.**

Система семантического кэширования предназначена для идентификации похожих или идентичных запросов пользователей. При обнаружении соответствующего запроса система извлекает соответствующую информацию из кэша, что сокращает необходимость в ее извлечении из исходного источника.

Поскольку при сравнении учитывается семантическое значение запросов, они не обязательно должны быть идентичными, чтобы система распознала их как один и тот же вопрос.  Они могут быть сформулированы по-разному или содержать неточности, будь то типографские или в структуре предложения, и мы можем определить, что пользователь действительно запрашивает ту же информацию.

Например, такие запросы, как **Какая столица Франции?**, **Скажите, как называется столица Франции?** и **Что такое столица Франции?**, имеют одинаковую цель и должны быть идентифицированы как один и тот же вопрос.

Хотя ответ модели может отличаться в зависимости от запроса краткого ответа во втором примере, информация, полученная из базы данных vector, должна быть такой же. Вот почему я размещаю систему кэширования между пользователем и базой данных vector, а не между пользователем и большой языковой моделью.

<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/semantic_cache.jpg?raw=true">

Большинство руководств, которые поясняют, как создать систему RAG, предназначены для использования одним пользователем, то есть для работы в среде тестирования. Другими словами, в ноутбуке можно взаимодействовать с локальной векторной базой данных и выполнять вызовы API или использовать локально сохраненную модель.

Эта архитектура быстро становится недостаточной при попытке перевести одну из этих моделей в рабочую среду, где они могут столкнуться с десятками или тысячами повторяющихся запросов.

Одним из способов повышения производительности является использование одного или нескольких семантических кэшей. В этом кэше сохраняются результаты предыдущих запросов, и перед обработкой нового запроса он проверяет, был ли ранее получен аналогичный запрос. Если это так, то вместо повторного выполнения процесса он извлекает информацию из кэша.

В системе RAG есть два момента, которые требуют много времени:

Извлечение информации, используемой для создания расширенного запроса:
Вызовите большую языковую модель для получения ответа.
В обеих точках может быть реализована система семантического кэширования, и мы могли бы даже иметь два кэша, по одному для каждой точки.

Размещение его в точке отклика модели может привести к потере влияния на полученный ответ. Наша система кэширования может рассматривать "Объясните Французскую революцию в 10 словах" и "Объясните Французскую революцию в ста словах" как один и тот же запрос. Если наша система кэширования хранит ответы моделей, пользователи могут подумать, что их инструкции выполняются неточно.

Но для обоих запросов потребуется одна и та же информация, чтобы обогатить запрос. Это основная причина, по которой я решил разместить систему семантического кэширования между запросом пользователя и извлечением информации из базы данных vector.

Однако это только одно из возможных решений. В зависимости от типа ответов и системных запросов, он может быть размещен в той или иной точке. Очевидно, что кэширование ответов модели позволило бы максимально сэкономить время, это происходит за счет потери влияния пользователя на ответ.


# Импортируйте и загрузите библиотеки.
Для начала нам нужно установить необходимые пакеты Python.
* **[sentence transformers](http:/www.sbert.net/)**. Эта библиотека необходима для преобразования предложений в векторы фиксированной длины, также известные как встраивания.
* **[xformers](https://github.com/facebookresearch/xformers)**. это пакет, который предоставляет библиотеки и утилиты для облегчения работы с моделями-трансформерами. Нам необходимо установить его, чтобы избежать ошибок при работе с моделью и внедрениями.  
* **[chromadb](https://www.trychroma.com/)**. Это наша векторная база данных. ChromaDB проста в использовании и имеет открытый исходный код, возможно, это наиболее часто используемая векторная база данных, используемая для хранения вложений.
* **[accelerate](https://github.com/huggingface/accelerate)** Необходимо запустить модель на графическом процессоре.

In [1]:
!pip install -q transformers==4.38.1
!pip install -q accelerate==0.27.2
!pip install -q sentence-transformers==2.5.1
!pip install -q xformers==0.0.24
!pip install -q chromadb==0.4.24
!pip install -q datasets==2.17.1

In [2]:
import numpy as np
import pandas as pd

# Загрузка датасета

In [3]:
from datasets import load_dataset

data = load_dataset("lmqg/qag_ruquad", split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for lmqg/qag_ruquad contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/lmqg/qag_ruquad
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings

In [4]:
data = data.to_pandas()
data["id"]=data.index
data.head(10)

,answers,questions,paragraph,questions_answers,id
0,"[При нулевом, Everybody , Warner Bros, чтобы ...",[При каком бюджете на раскрутку фото певицы ре...,"Everybody , как и хотела Мадонна, выпускают с...",question: При каком бюджете на раскрутку фото ...,0
1,"[Дилана, Кумиры британской публики The Beatles...",[По чьему примеру битлы стали экспериментирова...,Freewheelin’ Bob Dylan произвёл большое впеча...,question: По чьему примеру битлы стали экспери...,1
2,"[Манитобский театр юного зрителя, Театральный ...","[Как называется первый англоязычный театр, пол...",Le Cercle Molière — старейший (основан в 1925...,question: Как называется первый англоязычный т...,2
3,"[около 50 персонажей, древние философы, в обра...",[Сколько персонажей на фреске Афинская школа Р...,Афинская школа — блестяще выполненная многофи...,question: Сколько персонажей на фреске Афинска...,3
4,"[самодисциплины, освобождение ума и разума от ...",[Посредством чего можно усмирить пламя желаний...,"Бхагавадгита описывает йогу как контроль ума,...",question: Посредством чего можно усмирить плам...,4
5,"[Котиаион, укрепление, В этой, римские воины, ...",[Как на греческом языке в то время называлось ...,В этой области много многолюдных поселков. Из...,question: Как на греческом языке в то время на...,5
6,"[1986, Лабиринт , Джокера, Понтия Пилата, 1988]",[В каком году вышел рок-музыкальный фильм Абсо...,"Весёлого Рождества, мистер Лоуренс произвёл в...",question: В каком году вышел рок-музыкальный ф...,6
7,"[с военной жизнью, Земледелие, Георгики , ате...",[С чем часто сравнивает поэт подробности земле...,Георгики считаются самым совершенным произвед...,question: С чем часто сравнивает поэт подробно...,7
8,"[власть воли, состояние комы , это замкнутый к...","[Какое понятие ввел А.Ф. Самойлов, Какое состо...",Жизнь — это замкнутый круг рефлекторной деяте...,"question: Какое понятие ввел А.Ф. Самойлов, an...",8
9,"[два сезона, Star Trek: The Animated Series]",[Сколько сезонов включает в себя Звёздный путь...,Звёздный путь: Анимационный сериал (англ. Sta...,question: Сколько сезонов включает в себя Звёз...,9


In [5]:
data = data.head(1000)

In [6]:
data["answers"] = data["answers"].apply(lambda x: x[0])
data["questions"] = data["questions"].apply(lambda x: x[0])

<ipython-input-6-725e95b9e414>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["answers"] = data["answers"].apply(lambda x: x[0])
<ipython-input-6-725e95b9e414>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["questions"] = data["questions"].apply(lambda x: x[0])


In [7]:
data.head(10)

,answers,questions,paragraph,questions_answers,id
0,При нулевом,При каком бюджете на раскрутку фото певицы реш...,"Everybody , как и хотела Мадонна, выпускают с...",question: При каком бюджете на раскрутку фото ...,0
1,Дилана,По чьему примеру битлы стали экспериментироват...,Freewheelin’ Bob Dylan произвёл большое впеча...,question: По чьему примеру битлы стали экспери...,1
2,Манитобский театр юного зрителя,"Как называется первый англоязычный театр, полу...",Le Cercle Molière — старейший (основан в 1925...,question: Как называется первый англоязычный т...,2
3,около 50 персонажей,Сколько персонажей на фреске Афинская школа Ра...,Афинская школа — блестяще выполненная многофи...,question: Сколько персонажей на фреске Афинска...,3
4,самодисциплины,Посредством чего можно усмирить пламя желаний ...,"Бхагавадгита описывает йогу как контроль ума,...",question: Посредством чего можно усмирить плам...,4
5,Котиаион,Как на греческом языке в то время называлось у...,В этой области много многолюдных поселков. Из...,question: Как на греческом языке в то время на...,5
6,1986,В каком году вышел рок-музыкальный фильм Абсол...,"Весёлого Рождества, мистер Лоуренс произвёл в...",question: В каком году вышел рок-музыкальный ф...,6
7,с военной жизнью,С чем часто сравнивает поэт подробности землед...,Георгики считаются самым совершенным произвед...,question: С чем часто сравнивает поэт подробно...,7
8,власть воли,Какое понятие ввел А.Ф. Самойлов,Жизнь — это замкнутый круг рефлекторной деяте...,"question: Какое понятие ввел А.Ф. Самойлов, an...",8
9,два сезона,Сколько сезонов включает в себя Звёздный путь:...,Звёздный путь: Анимационный сериал (англ. Sta...,question: Сколько сезонов включает в себя Звёз...,9


In [33]:
MAX_ROWS = 1000
DOCUMENT="questions"
TOPIC="answers"


**ChromaDB** требует, чтобы данные имели уникальный идентификатор. Мы можем сделать это с помощью этой инструкции, которая создаст новый столбец с именем **Id**.

# Импорт и настройка векторной БД
Я собираюсь использовать ChromaDB, самую популярную векторную базу данных с открытым исходным кодом.

Сначала нам нужно импортировать ChromaDB, а затем импортировать класс **Settings** из модуля **chromadb.config**. Этот класс позволяет нам изменять настройки системы ChromaDB и настраивать ее поведение.

In [34]:
import chromadb
from chromadb.config import Settings

Теперь нам нужно только указать путь, по которому будет храниться векторная база данных.

In [35]:
chroma_client = chromadb.PersistentClient(path="/VectorDB")

## Заполнение базы данных ChromaDB и запрос к ней
Данные в ChromaDB хранятся в виде коллекций. Если коллекция существует, нам нужно ее удалить.

В следующих строках мы создаем коллекцию, вызывая функцию ***create_collection*** в ***chroma_client***, созданную выше.

In [36]:
collection_name = "collection"
if len(chroma_client.list_collections()) > 0 and collection_name in [chroma_client.list_collections()[0].name]:
        chroma_client.delete_collection(name=collection_name)

collection = chroma_client.create_collection(name=collection_name)


Пришло время добавить данные в коллекцию. Используя функцию ***add***, мы должны сообщить, по крайней мере, ***documents***, ***metadatas*** и ***ids***.
* В **documents** мы храним большой текст, это отдельный столбец в каждом наборе данных.
* В **metadatas** отображаются вопросы, на которые дан ответ.
* В поле **ids** нам нужно указать уникальный идентификатор для каждой строки. Он ДОЛЖЕН быть уникальным! Я создаю идентификатор, используя диапазон MAX_ROWS.

In [37]:
collection.add(
    documents=data[DOCUMENT].tolist(),
    metadatas=[{TOPIC: topic} for topic in data[TOPIC].tolist()],
    ids=[f"id{x}" for x in range(MAX_ROWS)],
)

Как только у нас будет информация в базе данных, мы сможем запросить ее и получить данные, которые соответствуют нашим потребностям. Поиск выполняется внутри содержимого документа, и не требуется искать точное слово или фразу. Результаты будут основаны на сходстве между условиями поиска и содержанием документов.

Метаданные не используются при поиске, но их можно использовать для фильтрации или уточнения результатов после первоначального поиска.

Давайте определим функцию для запроса к базе данных Chroma DB.

In [38]:
def query_database(query_text, n_results=10):
    results = collection.query(query_texts=query_text, n_results=n_results )
    return results

## Создаем систему семантического кэширования
Для реализации системы кэширования мы будем использовать Faiss, библиотеку, которая позволяет сохранять вложения в памяти. Это очень похоже на то, что делает Chrome, но без ее персистентности.

Для этой цели мы создадим класс под названием semantic_cache, который будет работать со своим собственным кодировщиком и предоставлять пользователю необходимые функции для выполнения запросов.

В этом классе первый запрос завершается ошибкой (кэш), и если возвращаемые результаты превышают указанный порог, он возвращает результат из кэша. В противном случае он извлекает результат из базы данных Chroma.

Кэш хранится в файле .json.

In [39]:
!pip install -q faiss-cpu==1.8.0

In [40]:
import faiss
from sentence_transformers import SentenceTransformer
import time
import json

Эта функция инициализирует семантический кэш.

Она использует индекс Flats, который, возможно, не самый быстрый, но идеально подходит для небольших наборов данных. В зависимости от характеристик данных, предназначенных для кэширования, и ожидаемого размера набора данных можно использовать другой индекс, такой как HNSW или IVF.

In [41]:
def init_cache():
  index = faiss.IndexFlatL2(312)
  if index.is_trained:
            print('Index trained')

  # Initialize Sentence Transformer model
  encoder = SentenceTransformer('cointegrated/rubert-tiny2')

  return index, encoder

В функции `retrieve_cache` файл .json извлекается с диска на случай, если возникнет необходимость повторно использовать кэш в разных сеансах.

In [78]:
def retrieve_cache(json_file):
      try:
          with open(json_file, 'r') as file:
              cache = json.load(file)
      except FileNotFoundError:
          cache = {'questions': [], 'embeddings': [], 'answers': [], 'response_text': []}

      return cache

Функция `store_cache` сохраняет файл, содержащий данные кэша, на диск.

In [43]:
def store_cache(json_file, cache):
  with open(json_file, 'w') as file:
        json.dump(cache, file)

Эти функции будут использоваться в классе "Семантический кэш", который включает в себя функцию поиска и ее функцию инициализации.

Несмотря на то, что метод **ask** содержит значительный объем кода, ее назначение довольно простое. Он ищет в кэше вопрос, наиболее близкий к тому, который только что задал пользователь.

Затем проверяет, соответствует ли он указанному пороговому значению. В случае положительного результата он напрямую возвращает ответ из кэша; в противном случае он вызывает функцию `query_database` для извлечения данных из ChromaDB.

Я использовал евклидово расстояние вместо косинуса, который широко используется при векторных сравнениях. Этот выбор основан на том факте, что евклидово расстояние является метрикой по умолчанию, используемой Faiss. Хотя косинусоидальное расстояние также можно рассчитать, это усложняет задачу, которая может существенно не повлиять на конечный результат.

In [44]:
print(collection)

name='collection' id=UUID('6e5d78d7-089c-43a1-a0e6-bfdebf0b670c') metadata=None tenant='default_tenant' database='default_database'


In [79]:
class semantic_cache:
  def __init__(self, json_file="cache_file.json", thresold=0.35):
      # Initialize Faiss index with Euclidean distance
      self.index, self.encoder = init_cache()

      # Set Euclidean distance threshold
      # a distance of 0 means identicals sentences
      # We only return from cache sentences under this thresold
      self.euclidean_threshold = thresold

      self.json_file = json_file
      self.cache = retrieve_cache(self.json_file)

  def ask(self, question: str) -> str:
      # Method to retrieve an answer from the cache or generate a new one
      start_time = time.time()
      try:
          #First we obtain the embeddings corresponding to the user question
          embedding = self.encoder.encode([question])

          # Search for the nearest neighbor in the index
          self.index.nprobe = 8
          D, I = self.index.search(embedding, 1)

          if D[0] >= 0:
              if I[0][0] >= 0 and D[0][0] <= self.euclidean_threshold:
                  row_id = int(I[0][0])

                  print('Answer recovered from Cache. ')
                  print(f'{D[0][0]:.3f} smaller than {self.euclidean_threshold}')
                  print(f'Found cache in row: {row_id} with score {D[0][0]:.3f}')
                  print(f'response_text: ' + self.cache['response_text'][row_id])

                  end_time = time.time()
                  elapsed_time = end_time - start_time
                  print(f"Time taken: {elapsed_time:.3f} seconds")
                  return self.cache['response_text'][row_id]

          # Handle the case when there are not enough results
          # or Euclidean distance is not met, asking to chromaDB.
          answer  = query_database([question], 1)
          print(answer)
          response_text = answer['metadatas'][0][0]['answers']

          self.cache['questions'].append(question)
          self.cache['embeddings'].append(embedding[0].tolist())
          self.cache['answers'].append(answer)
          self.cache['response_text'].append(response_text)

          print('Answer recovered from ChromaDB. ')
          print(f'response_text: {response_text}')

          self.index.add(embedding)
          store_cache(self.json_file, self.cache)
          end_time = time.time()
          elapsed_time = end_time - start_time
          print(f"Time taken: {elapsed_time:.3f} seconds")

          return response_text
      except Exception as e:
          raise RuntimeError(f"Error during 'ask' method: {e}")

### Testing the semantic_cache class.

In [80]:
# Initialize the cache.
cache = semantic_cache()

Index trained


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [81]:
results = cache.ask("Какое понятие ввел А.Ф. Самойлов")

{'ids': [['id8']], 'distances': [[0.0]], 'metadatas': [[{'answers': 'власть воли'}]], 'embeddings': None, 'documents': [['Какое понятие ввел А.Ф. Самойлов']], 'uris': None, 'data': None}
Answer recovered from ChromaDB. 
response_text: власть воли
Time taken: 0.157 seconds


Как и ожидалось, этот ответ был получен из Chroma DB. Затем класс сохраняет его в кэше.

Теперь, если мы отправим второй вопрос, который будет совершенно другим, ответ также должен быть получен из ChromaDB. Это происходит потому, что сохраненный ранее вопрос настолько непохож на заданный, что он превысил бы указанный порог с точки зрения евклидова расстояния.

In [82]:
results = cache.ask("При каком бюджете на раскрутку фото певицы решают не помещать на обложке ?")

{'ids': [['id0']], 'distances': [[0.0]], 'metadatas': [[{'answers': 'При нулевом'}]], 'embeddings': None, 'documents': [['При каком бюджете на раскрутку фото певицы решают не помещать на обложке ?']], 'uris': None, 'data': None}
Answer recovered from ChromaDB. 
response_text: При нулевом
Time taken: 0.238 seconds


In [83]:
question_def = "Чего лишен искусственный мёд по сравнению с натуральным?"
results = cache.ask(question_def)

{'ids': [['id11']], 'distances': [[0.0]], 'metadatas': [[{'answers': 'не имеет ферментов и не обладает ароматом'}]], 'embeddings': None, 'documents': [['Чего лишен искусственный мёд по сравнению с натуральным?']], 'uris': None, 'data': None}
Answer recovered from ChromaDB. 
response_text: не имеет ферментов и не обладает ароматом
Time taken: 0.221 seconds


In [84]:
question_def = "Чего лишен мёд по сравнению с натуральным?"
results = cache.ask(question_def)

Answer recovered from Cache. 
0.049 smaller than 0.35
Found cache in row: 2 with score 0.049
response_text: не имеет ферментов и не обладает ароматом
Time taken: 0.011 seconds


Отлично, система семантического кэширования работает так, как ожидалось.

Давайте протестируем ее с помощью вопроса, очень похожего на тот, который мы только что задали.

В этом случае ответ должен прийти непосредственно из кэша без необходимости обращаться к базе данных Chrome DB.

# Заключение.

При обращении к ChromaDB и непосредственном обращении к кэшу производительность повышается примерно на 50%. Однако в более крупных проектах эта разница увеличивается, что приводит к повышению производительности на 90-95%.

Тут очень мало данных в ChromaDB и только один экземпляр класса cache. Как правило, объем данных, хранящихся в системе кэширования, намного больше, и, возможно, это не просто запрос к базе данных vector, но они получены из разных источников.

Обычно существует несколько экземпляров класса cache, обычно основанных на типологии пользователей, поскольку вопросы, как правило, чаще повторяются среди пользователей, имеющих общие черты.

Таким образом, создана простая система RAG (поисково-расширенной генерации) и дополнена слоем семантического кэша между вопросом пользователя и получением информации, необходимой для создания расширенного запроса.